In [20]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

sparqlwd = SPARQLWrapper("https://query.wikidata.org/sparql")
myid = "wd:Q22673982"

query = """
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q891723. # doit avoir comme nature 'public company'
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
}
"""

sparqlwd.setQuery(query)
sparqlwd.setReturnFormat(JSON)
results = sparqlwd.query().convert()

In [21]:
liste_public_company = []
for r in results['results']['bindings']:
    liste_public_company.append({'name': r['itemLabel']['value'], 'source': 'wikidata', 'type': 'public company'})
df_public_company = pd.DataFrame(liste_public_company)

# scanR

In [36]:
import ast

In [22]:
df_scanr = pd.read_csv('https://data.enseignementsup-recherche.gouv.fr/api/explore/v2.1/catalog/datasets/export-des-organisations-exposees-dans-scanr/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B', sep=';')

In [38]:
def parse(x):
    try:
        return ast.literal_eval(x)
    except:
        return None

In [43]:
df_entreprises_scanr = df_scanr[(df_scanr.isFrench==1) & (df_scanr.kind=='Secteur privé')]

df_entreprises_scanr


df_entreprises_scanr_badge = df_entreprises_scanr[df_entreprises_scanr.badges.apply(lambda x:isinstance(get_list(x), list))]

In [51]:
df_entreprises_scanr_badge.head().label.apply(lambda x:x['default'])

TypeError: string indices must be integers

In [52]:
df_entreprises_scanr_badge.head().label.values[0]

'{"fr": "GENINDEXE", "default": "GENINDEXE"}'